In [ ]:
class SimpleSQLQueryBuilder:
    def __init__(self, table):
        self.table = table
        self.aggregate_columns = []
        self.dimension_columns = []
        self.where_clause = ""
        self.order_by_clause = ""
        self.limit_clause = ""

    def select_aggregate(self, aggregate_dict):
        for agg_type, columns in aggregate_dict.items():
            for col in columns:
                agg_col_name = f'"{agg_type.lower()}_{col.lower()}"'
                self.aggregate_columns.append(f"{agg_type.lower()}(\"{col}\") as {agg_col_name}")
        return self

    def select_dimension(self, dimension_columns):
        self.dimension_columns = [f'"{col}"' for col in dimension_columns]
        return self

    def where(self, condition):
        self.where_clause = " where " + condition
        return self

    def order_by(self, order_list, desc_columns=None):
        if desc_columns is None:
            desc_columns = set()

        order_clauses = []
        for column in order_list:
            order = "desc" if column in desc_columns else "asc"
            order_clauses.append(f'"{column}" {order}')
        self.order_by_clause = " order by " + ", ".join(order_clauses)
        return self

    def limit(self, number):
        self.limit_clause = " limit " + str(number)
        return self

    def build(self):
        all_columns = self.dimension_columns + self.aggregate_columns
        columns_str = ", ".join(all_columns)
        query = f"select {columns_str} from \"{self.table}\""
        if self.where_clause:
            query += self.where_clause
        if self.dimension_columns:
            group_by_str = ", ".join(self.dimension_columns)
            query += " group by " + group_by_str
        if self.order_by_clause:
            query += self.order_by_clause
        if self.limit_clause:
            query += self.limit_clause
        return query

    @property
    def selected_columns(self):
        return ", ".join(self.dimension_columns + self.aggregate_columns)

    @property
    def where_component(self):
        return self.where_clause

    @property
    def order_by_component(self):
        return self.order_by_clause

    @property
    def limit_component(self):
        return self.limit_clause

# 使用例
query_builder = SimpleSQLQueryBuilder("users")
query = query_builder.select_aggregate({"sum": ["Salary"], "max": ["Age"]}).select_dimension(["Department"]).where("age > 30").order_by(["department", "age"], desc_columns={"age"}).limit(10).build()

# クエリとそのコンポーネントを表示
print("Selected Columns:", query_builder.selected_columns)
print("Where Clause:", query_builder.where_component)
print("Order By Clause:", query_builder.order_by_component)
print("Limit Clause:", query_builder.limit_component)
print("\nComplete Query:", query)
